# Dense Retrieval with Simple Transformers - Training


## Imports and logging

In [22]:
import logging
import json

from simpletransformers.retrieval import RetrievalModel, RetrievalArgs
import wandb


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

2022-08-23 15:14:35.128017: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


## Data Preparation

The dataset can be downloaded from https://github.com/facebookresearch/DPR

```bash
# Downloads the dataset from https://github.com/facebookresearch/DPR
python download_data.py --resource data.retriever.nq-train
python download_data.py --resource data.retriever.nq-dev

# Move things around and clean up
mv downloads/data .
mv data/retriever/* data
rm -r data/retriever
rm -r downloads
```

In [2]:
train_data = "data/nq-train.json"
eval_data = "data/nq-dev.json"

In [5]:
with open(train_data, "r") as f:
    train = json.load(f)

## Data Formats

### DPR format

A JSON file where each entry is a dictionary must contain the two keys:
- `question`
- `postive_ctxs`

In [6]:
train[0].keys()

dict_keys(['dataset', 'question', 'answers', 'positive_ctxs', 'negative_ctxs', 'hard_negative_ctxs'])

Here, `positive_ctxs` is a list of relevant documents where each entry must contain the two keys:
- `title`
- `text`

While this list may contain multiple relevant documents, we only use the first (most) relevant document during training

In [7]:
train[0]["positive_ctxs"][0].keys()

dict_keys(['title', 'text', 'score', 'title_score', 'passage_id'])

Overall, Simple Transformers looks for a query and the relevant doc/passage for that query. The relevant passage may also contain an optional `title` value.

In [11]:
print(f"Query: {train[0]['question']}")
print()
print(f"Relevant passage title: {train[0]['positive_ctxs'][0]['title']}")
print()
print(f"Relevant passage title: {train[0]['positive_ctxs'][0]['text']}")


Query: big little lies season 2 how many episodes

Relevant passage title: Big Little Lies (TV series)

Relevant passage title: series garnered several accolades. It received 16 Emmy Award nominations and won eight, including Outstanding Limited Series and acting awards for Kidman, Skarsgård, and Dern. The trio also won Golden Globe Awards in addition to a Golden Globe Award for Best Miniseries or Television Film win for the series. Kidman and Skarsgård also received Screen Actors Guild Awards for their performances. Despite originally being billed as a miniseries, HBO renewed the series for a second season. Production on the second season began in March 2018 and is set to premiere in 2019. All seven episodes are being written by Kelley


## Recommended data format for custom datasets

### TSV file

The recommended data format for custom datasets is a simple TSV file with the following 3 columns:
- `query_text`
- `gold_passage`
- `title`

Alternatively, a Pandas Dataframe with the same columns may be used.

## Retrieval Model

The retrieval model is a dual encoder consisting of two BERT encoders

In [2]:
model_type = "custom"
model_name = None
context_name = "bert-base-uncased"
question_name = "bert-base-uncased"

In [4]:
model_args = RetrievalArgs()

# Training parameters
model_args.num_train_epochs = 40
model_args.train_batch_size = 40
model_args.learning_rate = 1e-5
model_args.max_seq_length = 256

# Evaluation parameters
model_args.retrieve_n_docs = 100
model_args.eval_batch_size = 100
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
# model_args.evaluate_during_training_steps = 200

# Model tracking
model_args.wandb_project = "Dense retrieval with Simple Transformers"
model_args.save_model_every_epoch = False
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.save_best_model = True
model_args.overwrite_output_dir = True

In [5]:
model = RetrievalModel(
    model_type=model_type,
    model_name=model_name,
    context_encoder_name=context_name,
    query_encoder_name=question_name,
    args=model_args,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.we

## Training the model

To train the model, we simply call `train_model` and pass the path to the training data file, along with the validation data file (required for evaluating during training).

In [6]:
model.train_model(train_data, eval_data=eval_data)

Generating train split:   0%|          | 0/58880 [00:00<?, ? examples/s]

Dataset retrieval_dataset_loading_script downloaded and prepared to /deep_learning/.cache/huggingface/datasets/retrieval_dataset_loading_script/default-e40ee4a6467d4bb8/0.0.0/f0c8460ab8d4db814fa43c3f14f6e1ad1e59a0b97758c518dafbf7dcd9e00fc3. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

/home/thilina/miniconda3/envs/st/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
INFO:simpletransformers.retrieval.retrieval_model: Training started


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.13.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-08-18 13:16:11.500419: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


INFO:simpletransformers.retrieval.retrieval_model:{'eval_loss': 24.18571879646995, 'mrr@1': 0.6581734458940905, 'mrr@2': 0.7214121258633922, 'mrr@3': 0.7382962394474291, 'mrr@5': 0.7491864927091327, 'mrr@10': 0.7545236267953075, 'top_1_accuracy': 0.6581734458940905, 'top_2_accuracy': 0.7846508058326938, 'top_3_accuracy': 0.8353031465848043, 'top_5_accuracy': 0.8825786646201075, 'top_10_accuracy': 0.9215656178050652}


Running Epoch 3 of 5:   0%|          | 0/1472 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Tracking training progress

In [1]:
import wandb

In [2]:
wandb.login()

wandb: Currently logged in as: thilina. Use `wandb login --relogin` to force relogin


True

In [3]:
api = wandb.Api()
run_path = "thilina/Dense retrieval with Simple Transformers/3k77hhta"

run = api.run(run_path)

### Training losses at each logging step

In [4]:
run.history().dropna(subset=["Training loss"])

,_step,_runtime,lr,Training loss,global_step,_timestamp,top_1_accuracy,mrr@1,train_loss,top_3_accuracy,...,gradients/graph_1embeddings.word_embeddings.weight,gradients/encoder.layer.6.attention.self.query.bias,gradients/encoder.layer.8.attention.output.dense.bias,gradients/graph_1encoder.layer.7.attention.self.query.bias,gradients/embeddings.word_embeddings.weight,gradients/encoder.layer.6.attention.self.query.weight,gradients/graph_1encoder.layer.11.attention.output.LayerNorm.bias,gradients/graph_1encoder.layer.6.output.LayerNorm.weight,gradients/graph_1encoder.layer.6.output.dense.bias,gradients/encoder.layer.4.attention.self.key.bias
0,6,174,9.906595e-07,9.326607e+00,350.0,1660863692,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15,391,2.264365e-06,3.730740e+00,800.0,1660863909,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,415,2.405887e-06,3.454845e+00,850.0,1660863933,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17,439,2.547410e-06,2.934197e+00,900.0,1660863957,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,463,2.688933e-06,2.701782e+00,950.0,1660863981,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,1299,40719,1.228612e-07,4.877671e-03,58200.0,1660904237,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510,1302,40791,9.575948e-08,7.152554e-08,58350.0,1660904309,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,1303,40815,8.672557e-08,1.490113e-07,58400.0,1660904333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,1305,40863,6.865774e-08,9.651372e-06,58500.0,1660904381,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Evaluation during training

In [18]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 100em; }</style>"))

In [20]:
%wandb thilina/Dense%20retrieval%20with%20Simple%20Transformers -h 1024